<div class="alert alert-success"><b>
Data Collection from websites
</div>

# Property listings

In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

%matplotlib inline

#### Access Houses in NBO webpage 

In [ ]:
r = requests.get("https://house.jumia.co.ke/for-sale/")
c = r.content
soup = BeautifulSoup(c, "html.parser")
# check source code of loaded page
# print(soup.prettify())

# # extract division
all = soup.find_all("div",{"class":"listing-info"})
len(all)

#### check  price element 

In [ ]:
price = all[0].find("span",{"class":"listing-price"}).text.replace(",", "")
price

## Data Collection 

        - Crawl web pages 
        - Extract selected elements 
        - Colect dict and store in a csv

In [ ]:
base_url = "https://house.jumia.co.ke/nairobi/house/for-sale/?page="
data = []
#   crawl though available pages to extract page no. links
for page in range(1,51,1): #used 51 since its the last page
    prop_page = base_url+str(page)+str("&size=30")
    r = requests.get(prop_page)
    c = r.content
    soup = BeautifulSoup(c, "html.parser")
    all = soup.find_all("div",{"class":"listing-info"})
    # collect price, detail and size from each page
    for item in all:
        d={}
    #  property value
        d["value"] = item.find("span",{"class":"listing-price"}).text.replace(",","")
    #   property description 
        d["desc"] = (item.find("a",{"class":"main-link"}).text).strip() 
    #   property size / bedroom capacity
        try:
             d["size"] = item.find("span",{"class":"listing-attributes-value"}).text.replace("\n","")
        except:
            d["size"] = None
    #   property address
        d["location"] = item.find("p",{"class":"listing-address icon-location"}).text
    
    #   collect data in a list
        data.append(d)
        
    #   store data in a dataframe
        df = pd.DataFrame(data)
    #   save to csv
        df.to_csv("scrapped_data/listing.csv")

## Load Data to DataFrame for cleaning

In [ ]:
df1 = pd.read_csv('scrapped_data/listing.csv',
                      usecols=['desc', 'location', 'size', 'value'])

df1.head()

#### clean columns

In [ ]:
# Save value as Numeric in Ksh
df1['value'] = pd.to_numeric(df1['value'].str.replace("Contact seller for price|~|KSh| ","0"))

# Size to no of Bedrooms
df1['size'] = df1['size'].str.split(' ').str[0]

df1.head()

In [ ]:
# save clean dataset
df1.to_csv('cleaned_data/NBO_listings_data.csv')

In [ ]:
# df1.drop(562,axis=0,inplace=True)
df1.shape

